In [1]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
from torchvision.utils import save_image

In [7]:
# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Create a directory if not exists
sample_dir = 'samples'
if not os.path.exists(sample_dir):
    os.makedirs(sample_dir)

# Hyper-parameters
image_size = 784
h_dim = 400
z_dim = 20
num_epochs = 15
batch_size = 128
learning_rate = 1e-3

In [8]:
# MNIST dataset
dataset = torchvision.datasets.MNIST(root='../data',train=True,transform=transforms.ToTensor(),download=True)
# Data Loader
data_loader = torch.utils.data.DataLoader(dataset=dataset,batch_size=batch_size,shuffle=True)

In [9]:
# VAE model
class VAE(nn.Module):
    def __init__(self, image_size=784, h_dim=400, z_dim=20):
        super(VAE, self).__init__()
        self.fc1 = nn.Linear(image_size, h_dim)
        self.fc2 = nn.Linear(h_dim, z_dim)
        self.fc3 = nn.Linear(h_dim, z_dim)
        self.fc4 = nn.Linear(z_dim, h_dim)
        self.fc5 = nn.Linear(h_dim, image_size)
        
    def encode(self, x):
        h = F.relu(self.fc1(x))
        return self.fc2(h), self.fc3(h)
    
    def reparameterize(self, mu, log_var):
        std = torch.exp(log_var/2)
        eps = torch.randn_like(std)
        return mu + eps * std

    def decode(self, z):
        h = F.relu(self.fc4(z))
        return F.sigmoid(self.fc5(h))
    
    def forward(self, x):
        mu, log_var = self.encode(x)
        z = self.reparameterize(mu, log_var)
        x_reconst = self.decode(z)
        return x_reconst, mu, log_var

model = VAE().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
# Start training
for epoch in range(num_epochs):
    for i, (x, _) in enumerate(data_loader):
        # Forward pass
        x = x.to(device).view(-1,image_size)
        x_reconst,mu,log_var = model(x)
        # Compute reconstruction loss and kl divergence
        # For KL divergence, see Appendix B in VAE paper or http://yunjey47.tistory.com/43
        reconst_loss = F.binary_cross_entropy(x_reconst,x,size_average=False)
        kl_div = -0.5 * torch.sum(1 + log_var-mu.pow(2)-log_var.exp())
        
        # Backprop and optimize
        loss = reconst_loss + kl_div
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 10 == 0:
            print ("Epoch[{}/{}], Step [{}/{}], Reconst Loss: {:.4f}, KL Div: {:.4f}" 
                   .format(epoch+1, num_epochs, i+1, len(data_loader), reconst_loss.item(), kl_div.item()))
    with torch.no_grad():
        # Save the sampled images
        z = torch.randn(batch_size,z_dim).to(device)
        out = model.decode(z).view(-1,1,28,28)
        save_image(out,os.path.join(sample_dir,'sampled-{}.png'.format(epoch+1)))
        
        # Save the reconstructed images
        out,_,_ = model(x)
        x_concat = torch.cat([x.view(-1,1,28,28),out.view(-1,1,28,28)],dim=3)
        save_image(x_concat,os.path.join(sample_dir,'reconst-{}.png'.format(epoch+1)))

Epoch[1/15], Step [10/469], Reconst Loss: 37527.1797, KL Div: 2215.7207
Epoch[1/15], Step [20/469], Reconst Loss: 29678.2109, KL Div: 1192.4722
Epoch[1/15], Step [30/469], Reconst Loss: 27221.8828, KL Div: 1219.1658
Epoch[1/15], Step [40/469], Reconst Loss: 26725.2773, KL Div: 679.0154
Epoch[1/15], Step [50/469], Reconst Loss: 26753.5859, KL Div: 731.0175
Epoch[1/15], Step [60/469], Reconst Loss: 24786.4688, KL Div: 939.4626
Epoch[1/15], Step [70/469], Reconst Loss: 24787.0977, KL Div: 866.4673
Epoch[1/15], Step [80/469], Reconst Loss: 24433.2773, KL Div: 864.0120
Epoch[1/15], Step [90/469], Reconst Loss: 23825.1484, KL Div: 1043.4161
Epoch[1/15], Step [100/469], Reconst Loss: 23211.9180, KL Div: 1318.6299
Epoch[1/15], Step [110/469], Reconst Loss: 21335.8711, KL Div: 1416.9479
Epoch[1/15], Step [120/469], Reconst Loss: 20779.1777, KL Div: 1731.3557
Epoch[1/15], Step [130/469], Reconst Loss: 19461.7969, KL Div: 1603.1835
Epoch[1/15], Step [140/469], Reconst Loss: 19236.0273, KL Div: 17